# MCP Weather Agent - Jupyter Notebook Version

This notebook demonstrates a LangChain agent that uses MCP (Model Context Protocol) tools instead of local tools.

## Setup
1. Make sure your MCP server is running
2. Set your OpenAI API key
3. Run the cells below

In [ ]:
# Install required packages
!pip install langchain langchain-openai python-dotenv fastmcp

In [ ]:
# Set your OpenAI API key
%env OPENAI_API_KEY=your-openai-api-key-here

In [ ]:
import os
import asyncio
import json
from typing import Dict, Any
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain_core.prompts import PromptTemplate
from fastmcp import Client

# Load environment variables
load_dotenv()

# Initialize the LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

# MCP Server URL 
MCP_SERVER_URL = "your-mcp-server-url"

print("✅ Imports and LLM initialized")

In [ ]:
# Simple synchronous MCP client for Jupyter
def call_mcp_tool_sync(tool_name: str, arguments: Dict[str, Any]) -> str:
    """Synchronous wrapper for MCP tool calls"""
    async def _call_mcp_tool():
        try:
            async with Client(MCP_SERVER_URL) as client:
                print(f"🛠️ Calling MCP tool: {tool_name} with args: {arguments}")
                result = await client.call_tool(tool_name, arguments)
                
                if result.content and len(result.content) > 0:
                    return result.content[0].text
                else:
                    return f"No result from {tool_name}"
        except Exception as e:
            return f"Error calling {tool_name}: {str(e)}"
    
    # Run the async function
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            # We're in Jupyter, create a new event loop in a thread
            import nest_asyncio
            nest_asyncio.apply()
            return loop.run_until_complete(_call_mcp_tool())
        else:
            return asyncio.run(_call_mcp_tool())
    except:
        # Fallback: create new event loop
        return asyncio.run(_call_mcp_tool())

print("✅ MCP client functions defined")

In [ ]:
# Install nest_asyncio for Jupyter compatibility
!pip install nest_asyncio

In [ ]:
# Discover MCP tools and create LangChain tools
async def discover_mcp_tools():
    """Discover available tools from MCP server"""
    try:
        async with Client(MCP_SERVER_URL) as client:
            tools = await client.list_tools()
            print(f"🔗 Connected to MCP server! Found {len(tools)} tools:")
            for tool in tools:
                print(f"   • {tool.name}: {tool.description}")
            return tools
    except Exception as e:
        print(f"❌ Failed to connect to MCP server: {e}")
        return []

# Run discovery
import nest_asyncio
nest_asyncio.apply()

mcp_tools = await discover_mcp_tools()

In [ ]:
# Create LangChain tools from MCP tools
def create_langchain_tool(mcp_tool):
    """Create a LangChain tool from an MCP tool"""
    def tool_function(tool_input: str) -> str:
        # Parse input - handle both JSON strings and plain strings
        try:
            if tool_input.startswith('{') and tool_input.endswith('}'):
                arguments = json.loads(tool_input)
            else:
                # Treat as location for weather tools
                arguments = {"location": tool_input.strip()}
        except json.JSONDecodeError:
            arguments = {"location": tool_input.strip()}
        
        return call_mcp_tool_sync(mcp_tool.name, arguments)
    
    return Tool(
        name=mcp_tool.name,
        description=mcp_tool.description or f"MCP tool: {mcp_tool.name}",
        func=tool_function
    )

# Create LangChain tools
langchain_tools = [create_langchain_tool(tool) for tool in mcp_tools]

print(f"✅ Created {len(langchain_tools)} LangChain tools from MCP server")

In [ ]:
# Create the LangChain agent
template = '''You are a helpful weather assistant powered by MCP (Model Context Protocol). 
You provide weather-related information using remote weather intelligence tools.

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

agent = create_react_agent(
    tools=langchain_tools,
    llm=llm,
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent, 
    tools=langchain_tools, 
    verbose=True,
    handle_parsing_errors=True
)

print("✅ MCP Weather Agent ready!")

## Test the MCP Weather Agent

Now you can use the agent just like in your example! Use `agent_executor.invoke()` to ask weather questions:

In [ ]:
# Test 1: Current weather
agent_executor.invoke({"input": "What is the weather in Oslo now?"})

In [ ]:
# Test 2: Weather forecast
agent_executor.invoke({"input": "Get tomorrow's weather forecast for Oslo."})